In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import random
from tqdm.notebook import tqdm
from geopy.distance import geodesic

In [2]:
df = pd.read_csv("London postcodes.csv", delimiter=',')

In [3]:
display(df)

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,...,Travel To Work Area,ITL level 2,ITL level 3,UPRNs,Distance to sea,LSOA21 Code,Lower layer super output area 2021,MSOA21 Code,Middle layer super output area 2021,Census output area 2021
0,BR1 1AA,Yes,51.401546,0.015415,540291,168873,TQ402688,Greater London,Bromley,Bromley Town,...,London,Outer London - South,Bromley,"10070014435,10070014436,10070014437,1007001443...",28.0730,E01034386,Bromley 018G,E02000144,Bromley South,E00182600
1,BR1 1AB,Yes,51.406333,0.015208,540262,169405,TQ402694,Greater London,Bromley,Bromley Town,...,London,Outer London - South,Bromley,"10070008860,10070008861,10070008862,1007000886...",27.9776,E01000676,Bromley 008B,E02000134,Bromley North & Sundridge,E00003255
2,BR1 1AD,No,51.400057,0.016715,540386,168710,TQ403687,Greater London,Bromley,Bromley Town,...,London,Outer London - South,Bromley,NaN,28.0211,E01034386,Bromley 018G,E02000144,Bromley South,E00182639
3,BR1 1AE,Yes,51.404543,0.014195,540197,169204,TQ401692,Greater London,Bromley,Bromley Town,...,London,Outer London - South,Bromley,"10003640209,10070000614,10070002658,1007000265...",28.0861,E01000677,Bromley 018C,E02000144,Bromley South,E00003266
4,BR1 1AF,Yes,51.401392,0.014948,540259,168855,TQ402688,Greater London,Bromley,Bromley Town,...,London,Outer London - South,Bromley,"10070014484,10070014485,10070014486,1007001448...",28.1083,E01034386,Bromley 018G,E02000144,Bromley South,E00182600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13477,CR2 2JN,No,51.341077,-0.119204,531100,161900,TQ311618,Greater London,Croydon,Purley & Woodcote,...,London,Outer London - South,Croydon,NaN,38.9074,E01001092,Croydon 037B,E02000230,Purley Central,E00181026
13478,CR2 2JP,No,51.340179,-0.119241,531100,161800,TQ310618,Greater London,Croydon,Purley & Woodcote,...,London,Outer London - South,Croydon,NaN,38.9428,E01001092,Croydon 037B,E02000230,Purley Central,E00181026
13479,CR2 2JQ,No,51.343796,-0.120529,531000,162200,TQ309621,Greater London,Croydon,Purley & Woodcote,...,London,Outer London - South,Croydon,NaN,38.8960,E01001092,Croydon 037B,E02000230,Purley Central,E00005455
13480,CR2 2JR,No,51.354719,-0.128699,530400,163400,TQ304634,Greater London,Sutton,South Beddington & Roundshaw,...,London,Outer London - South,"Merton, Kingston upon Thames and Sutton",NaN,39.0653,E01004088,Sutton 019C,E02000858,Roundshaw,E00020565


In [4]:
df.columns

Index(['Postcode', 'In Use?', 'Latitude', 'Longitude', 'Easting', 'Northing',
       'Grid Ref', 'County', 'District', 'Ward', 'District Code', 'Ward Code',
       'Country', 'County Code', 'Constituency', 'Introduced', 'Terminated',
       'Parish', 'National Park', 'Population', 'Households', 'Built up area',
       'Built up sub-division', 'Lower layer super output area', 'Rural/urban',
       'Region', 'Altitude', 'London zone', 'LSOA Code', 'Local authority',
       'MSOA Code', 'Middle layer super output area', 'Parish Code',
       'Census output area', 'Constituency Code',
       'Index of Multiple Deprivation', 'Quality', 'User Type', 'Last updated',
       'Nearest station', 'Distance to station', 'Postcode area',
       'Postcode district', 'Police force', 'Water company', 'Plus Code',
       'Average Income', 'Sewage Company', 'Travel To Work Area',
       'ITL level 2', 'ITL level 3', 'UPRNs', 'Distance to sea', 'LSOA21 Code',
       'Lower layer super output area 2021', '

In [5]:
rides = pd.DataFrame(columns=['driver_id', 'client_id',
                              'start', 'start_latitude', 'start_longtitude',
                              'finish', 'finish_latitude', 'finish_longtitude',
                              'distance', 'road_time', 'start_time', 'finish_time', 'cost',
                              'driver_rate', 'category_driver_feedback', 'text_driver_feedback',
                             'client_rate', 'category_client_feedback', 'text_client_feedback'])
NUM_RIDES = 1000000

Drivers and clients id's

In [6]:
rides['driver_id']=np.random.randint(low=0,high=3000,size=NUM_RIDES)
rides['client_id']=np.random.randint(low=0,high=5000,size=NUM_RIDES)

Start and finish points

In [7]:
rides[['start', 'start_latitude', 'start_longtitude']] = df[['Postcode', 'Latitude', 'Longitude']].sample(n=NUM_RIDES, replace=True).reset_index(drop=True)

In [8]:
rides[['finish', 'finish_latitude', 'finish_longtitude']] = df[['Postcode', 'Latitude', 'Longitude']].sample(n=NUM_RIDES, replace=True).reset_index(drop=True)

Start time

In [9]:
def random_dates(start, end, n=10):
    start_u = start.value//10**9
    end_u = end.value//10**9
    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

start = pd.to_datetime('2010-01-01')
end = pd.to_datetime('2020-01-01')
rides['start_time'] = random_dates(start, end, NUM_RIDES)

Distance between start and finish points

In [10]:
rides['distance'] = [geodesic((x1, y1), (x2, y2)).km for x1, y1, x2, y2 in tqdm(zip(rides['start_latitude'],
                                                                                    rides['start_longtitude'],
                                                                                    rides['finish_latitude'],
                                                                                    rides['finish_longtitude']), total=NUM_RIDES)]
rides['distance'] = rides['distance'].round(2)

  0%|          | 0/1000000 [00:00<?, ?it/s]

Calculate road time

In [11]:
rides['road_time'] = abs(np.random.normal(size=NUM_RIDES,scale=10)) + rides['distance'] * abs(np.random.normal(size=NUM_RIDES, loc=1, scale=0.2))
rides['road_time'] = rides['road_time'].astype('int')
rides['road_time'] = pd.to_timedelta(rides['road_time'], unit='m')

Calculate finish time

In [12]:
rides['finish_time'] = rides['start_time'] + rides['road_time']

Calculate cost of the ride

In [13]:
def count_cost(start_time, distance):
    cost = 3 + 0.5*distance
    if (start_time.hour>=8 and start_time.hour<=10) or \
        (start_time.hour>=17 and start_time.hour<=19):
        cost *= 1.5
    if (start_time.hour>=21 or start_time.hour<=6):
        cost *= 1.25
    return cost

rides['cost'] = [count_cost(s, d) for s, d in tqdm(zip(rides.start_time, rides.distance), total=NUM_RIDES)]
rides['cost'] = rides['cost'].round(2)

  0%|          | 0/1000000 [00:00<?, ?it/s]

Drivers rates

In [14]:
driver_rate_idx = np.random.randint(low=0, high=NUM_RIDES, size=int(NUM_RIDES*0.35))
driver_rate_distribution_arr = np.random.multinomial(1, [0.2, 0.05, 0.1, 0.25, 0.4], size=int(NUM_RIDES*0.35))
rides['driver_rate'][driver_rate_idx] = np.where(driver_rate_distribution_arr == 1)[1] + 1

In [15]:
driver_feedback_categories_good = ['great service', 'nice car', 'wonderful companion', 'neat and tidy', 'expert navigation', 'recommend']
driver_feedback_categories_bad = ['awful service', 'bad car', 'unpleasant companion', 'dirty', 'non-expert navigation', 'not recommend']

In [16]:
category_driver_good_feedback_idx = np.random.choice(rides[rides.driver_rate > 3].index, size=int(NUM_RIDES*0.3*0.2))
rides["category_driver_feedback"][category_driver_good_feedback_idx] = np.random.choice(driver_feedback_categories_good, size=int(NUM_RIDES*0.3*0.2))

category_driver_bad_feedback_idx = np.random.choice(rides[rides.driver_rate < 4].index, size=int(NUM_RIDES*0.3*0.2))
rides["category_driver_feedback"][category_driver_bad_feedback_idx] = np.random.choice(driver_feedback_categories_bad, size=int(NUM_RIDES*0.3*0.2))

In [17]:
text_good_feedback_driver_length = np.random.randint(low=0, high=7, size=int(NUM_RIDES*0.3*0.2))
text_good_feedback_driver_sample = [random.sample(driver_feedback_categories_good, i) for i in text_good_feedback_driver_length]
rides['text_driver_feedback'][category_driver_good_feedback_idx] = text_good_feedback_driver_sample

text_bad_feedback_driver_length = np.random.randint(low=0, high=7, size=int(NUM_RIDES*0.3*0.2))
text_bad_feedback_driver_sample = [random.sample(driver_feedback_categories_bad, i) for i in text_bad_feedback_driver_length]
rides['text_driver_feedback'][category_driver_bad_feedback_idx] = text_bad_feedback_driver_sample

Clients rates

In [18]:
client_rate_idx = np.random.randint(low=0,high=NUM_RIDES,size=int(NUM_RIDES*0.5))
client_rate_distribution_arr = np.random.multinomial(1, [0.2, 0.1, 0.15, 0.2, 0.35], size=int(NUM_RIDES*0.5))
rides['client_rate'][client_rate_idx] = np.where(client_rate_distribution_arr == 1)[1] + 1

In [19]:
client_feedback_categories_good = ['polite', 'pleasant', 'quiet', 'neat and tidy', 'recommend']
client_feedback_categories_bad = ['unpolite', 'unpleasant', 'loud', 'dirty','not recommend']

In [20]:
category_client_good_feedback_idx = np.random.choice(rides[rides.client_rate > 3].index, size=int(NUM_RIDES*0.06))
rides["category_client_feedback"][category_client_good_feedback_idx] = np.random.choice(client_feedback_categories_good, size=int(NUM_RIDES*0.3*0.2))

category_client_bad_feedback_idx = np.random.choice(rides[rides.client_rate < 4].index, size=int(NUM_RIDES*0.06))
rides["category_client_feedback"][category_client_bad_feedback_idx] = np.random.choice(client_feedback_categories_bad, size=int(NUM_RIDES*0.3*0.2))

In [21]:
text_good_feedback_client_length = np.random.randint(low=0, high=6, size=int(NUM_RIDES*0.3*0.2))
text_good_feedback_client_sample = [random.sample(client_feedback_categories_good, i) for i in text_good_feedback_client_length]
rides['text_client_feedback'][category_client_good_feedback_idx] = text_good_feedback_client_sample

text_bad_feedback_client_length = np.random.randint(low=0, high=6, size=int(NUM_RIDES*0.3*0.2))
text_bad_feedback_client_sample = [random.sample(client_feedback_categories_good, i) for i in text_bad_feedback_client_length]
rides['text_client_feedback'][category_client_good_feedback_idx] = text_bad_feedback_client_sample

In [22]:
display(rides)

,driver_id,client_id,start,start_latitude,start_longtitude,finish,finish_latitude,finish_longtitude,distance,road_time,start_time,finish_time,cost,driver_rate,category_driver_feedback,text_driver_feedback,client_rate,category_client_feedback,text_client_feedback
0,2116,627,BR5 3SF,51.393142,0.124063,BR3 3QR,51.392362,-0.040157,11.43,0 days 00:25:00,2010-11-11 02:53:56,2010-11-11 03:18:56,10.89,NaN,NaN,NaN,NaN,NaN,NaN
1,1361,4841,CR0 8HD,51.374676,-0.031199,CR0 8HN,51.375894,-0.029840,0.17,0 days 00:01:00,2012-11-14 04:05:23,2012-11-14 04:06:23,3.86,NaN,NaN,NaN,NaN,NaN,NaN
2,2964,2947,CR0 3EG,51.387221,-0.116522,BR7 6DP,51.422808,0.060247,12.92,0 days 00:16:00,2014-10-14 05:36:02,2014-10-14 05:52:02,11.82,NaN,NaN,NaN,NaN,NaN,NaN
3,2027,4356,BR4 0HA,51.389355,0.001023,BR1 3GY,51.408226,0.017578,2.39,0 days 00:07:00,2017-05-16 09:31:28,2017-05-16 09:38:28,6.29,5,NaN,NaN,5,NaN,NaN
4,398,4325,CR0 3RT,51.379295,-0.112953,BR2 8QD,51.371651,0.043556,10.93,0 days 00:15:00,2012-08-12 06:51:42,2012-08-12 07:06:42,10.58,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,2370,1338,CR2 1XA,51.338473,-0.125054,CR0 0YD,51.372605,-0.099622,4.19,0 days 00:12:00,2012-12-15 15:59:47,2012-12-15 16:11:47,5.10,NaN,NaN,NaN,NaN,NaN,NaN
999996,762,590,CR0 2SU,51.385388,-0.109383,CR0 1FX,51.369175,-0.098833,1.95,0 days 00:03:00,2015-12-29 02:29:59,2015-12-29 02:32:59,4.97,NaN,NaN,NaN,NaN,NaN,NaN
999997,2673,1400,BR3 1PQ,51.413914,-0.033483,BR1 5ZD,51.408226,0.017578,3.61,0 days 00:14:00,2017-03-07 21:22:50,2017-03-07 21:36:50,6.01,3,NaN,NaN,NaN,NaN,NaN
999998,2693,2535,BR5 4QR,51.376134,0.122241,CR2 0YL,51.341348,-0.080426,14.64,0 days 00:20:00,2019-10-21 13:04:19,2019-10-21 13:24:19,10.32,NaN,NaN,NaN,4,NaN,NaN


In [23]:
rides.to_csv("rides.csv")